**Task:**

Train and fine-tune a decision tree for the moons dataset by following these steps:
a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.

b. Use train_test_split() to split the dataset into a training set and a test set. 

c. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes. 

d. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy


a. Generate the moons dataset:

In [11]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)


b. Split the dataset into a training set and a test set:

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


c. Use grid search with cross-validation to find good hyperparameter values for a DecisionTreeClassifier:

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': [2, 4, 8, 16, 32, 64, 128]}
tree_clf = DecisionTreeClassifier(random_state=42)
grid_search_cv = GridSearchCV(tree_clf, params, cv=3, verbose=1)
grid_search_cv.fit(X_train, y_train)


Fitting 3 folds for each of 7 candidates, totalling 21 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 4, 8, 16, 32, 64, 128]},
             verbose=1)

d. Train the model on the full training set using the best hyperparameters and measure its performance on the test set:

In [14]:
from sklearn.metrics import accuracy_score

best_tree_clf = grid_search_cv.best_estimator_
y_pred = best_tree_clf.predict(X_test)
accuracy_score(y_test, y_pred)


0.863

**Accuracy: %86,3**

**Task:**

Grow a forest by following these steps: 
a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use ScikitLearn’s ShuffleSplit class for this. 

b. Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy. 

c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 decision trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set. 

d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a random forest classifier!

a. Generate 1,000 subsets of the training set:

In [15]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

shuffle_split = ShuffleSplit(n_splits=n_trees, test_size=n_instances, random_state=42)
subsets = []
for train_index, _ in shuffle_split.split(X_train):
    X_subset = X_train[train_index]
    y_subset = y_train[train_index]
    subsets.append((X_subset, y_subset))

b. Train one decision tree on each subset:

In [16]:
import numpy as np

forest = [DecisionTreeClassifier(max_leaf_nodes=best_tree_clf.max_leaf_nodes, random_state=42)
          for _ in range(n_trees)]
accuracies = []
for i in range(n_trees):
    forest[i].fit(subsets[i][0], subsets[i][1])
    accuracy = forest[i].score(X_test, y_test)
    accuracies.append(accuracy)

#print("Average accuracy of individual trees:", np.mean(accuracies))
np.mean(accuracies)

0.8633039999999997

Evaluate these 1,000 decision trees on the test set:

In [17]:
y_preds = []
for tree in forest:
    y_pred = tree.predict(X_test)
    y_preds.append(y_pred)
y_preds = np.array(y_preds)

accuracy = accuracy_score(y_test, y_pred)
print(f'Tree accuracy: {accuracy:.4f}')


Tree accuracy: 0.8630


c. Generate majority-vote predictions over the test set:

In [18]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(y_preds, axis=0)

C:\Users\Suat\AppData\Local\Temp\ipykernel_15392\4255047687.py:3: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_majority_votes, n_votes = mode(y_preds, axis=0)


d. Evaluate these predictions on the test set:

In [19]:
accuracy = np.sum(y_pred_majority_votes.ravel() == y_test) / len(y_test)
print("Accuracy:", accuracy)


Accuracy: 0.863
